DESCRIPCION

Actualizar la tabla de saldos mensuales 

========================================================================================

In [1]:
import mysql.connector
import configparser
import sys

In [2]:
def actualizar_saldo_mes(p_mes, p_anio):
    try:
        # Leer las credenciales del archivo de configuración
        config = configparser.ConfigParser()
        config.read('config.ini')

        user = config['database']['user']
        password = config['database']['password']
        host = config['database']['host']
        database = config['database']['database']

        # Abrir la base de datos: establecer la conexión
        conexion = mysql.connector.connect(user=user, password=password, host=host, database=database)

        # crear un cursor
        cursor = conexion.cursor()

        # =====================================================================================================================

        # Obtener el mes y año del periodo anterior
        if p_mes == 1:
            v_mes_anterior = 12
            v_anio_anterior = p_anio - 1
        else:
            v_mes_anterior = p_mes - 1
            v_anio_anterior = p_anio

        # Verificar si existe un registro para el mes anterior
        cursor.execute("SELECT COUNT(*) FROM tbl_saldo_mes WHERE mes = %s AND anio = %s", (v_mes_anterior, v_anio_anterior))
        v_existencia_mes_anterior = cursor.fetchone()[0]

        if v_existencia_mes_anterior == 0:
            print("No hay registro de saldos para el mes anterior. Por favor, verifique los parámetros o corrija la situación")
            sys.exit()

        # 1. Obtener la fecha del último día del mes
        cursor.execute("SELECT LAST_DAY('{}-{}-01')".format(p_anio, p_mes))
        v_fecha = cursor.fetchone()[0]

        # 2. Obtener el saldo inicial del periodo anterior
        cursor.execute("SELECT saldo_final FROM tbl_saldo_mes WHERE mes = %s AND anio = %s", (v_mes_anterior, v_anio_anterior))
        v_saldo_inicial = cursor.fetchone()[0]

        # 3. Obtener la sumatoria del campo NETO de tbl_transacciones
        cursor.execute("SELECT IFNULL(SUM(NETO), 0) FROM tbl_transacciones WHERE MES = %s AND ANIO = %s", (p_mes, p_anio))
        v_neto_mes = cursor.fetchone()[0]

        # 4. Calcular el saldo final
        v_saldo_final = v_saldo_inicial + v_neto_mes

        # 5. Verificar si ya existe un registro para este mes y año
        cursor.execute("SELECT COUNT(*) FROM tbl_saldo_mes WHERE mes = %s AND anio = %s", (p_mes, p_anio))
        v_existencia = cursor.fetchone()[0]

        if v_existencia > 0:
            # Actualizar el registro existente
            cursor.execute("UPDATE tbl_saldo_mes SET fecha = %s, saldo_inicial = %s, neto_mes = %s, saldo_final = %s WHERE mes = %s AND anio = %s", (v_fecha, v_saldo_inicial, v_neto_mes, v_saldo_final, p_mes, p_anio))
        else:
            # Insertar un nuevo registro
            cursor.execute("INSERT INTO tbl_saldo_mes (fecha, mes, anio, saldo_inicial, neto_mes, saldo_final) VALUES (%s, %s, %s, %s, %s, %s)", (v_fecha, p_mes, p_anio, v_saldo_inicial, v_neto_mes, v_saldo_final))

        # Confirmar los cambios
        conexion.commit()
        print("Proceso completado exitosamente.")

    except mysql.connector.Error as error:
        print("Error al conectar con la base de datos:", error)

    finally:
        # Cerrar la conexión
        if 'conexion' in locals() and conexion.is_connected():
            cursor.close()
            conexion.close()

# Ejemplo de uso
actualizar_saldo_mes(1, 2023)  # Especifica el mes y el año

Proceso completado exitosamente.
